In [ ]:
import os

In [ ]:
%pwd

'd:\\Text_summarization\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text_summarization'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True) #frozen=True makes the object immutable, like a constant.
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

#or
###
"""class DataIngestionConfig:
    def __init__(self, root_dir, source_URL, local_data_file, unzip_dir):
        self.root_dir = root_dir
        self.source_URL = source_URL
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir
"""

In [31]:
from src.TEXT_SUMMARIZATION.constants import *
from src.TEXT_SUMMARIZATION.common import read_yaml,create_directories


In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


    

In [33]:
import os
import urllib.request as request
import zipfile
from src.TEXT_SUMMARIZATION.logger import logging
from src.TEXT_SUMMARIZATION.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            """Headers contains info like:Server type,Content-type (application/zip),Content-length (file size),Date, cache info, etc."""

            logging.info(f"{filename} download with following info: \n{headers}")
        else:
            logging.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [41]:
from src.TEXT_SUMMARIZATION.exception import CustomException
import sys
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)